In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
from models.Flexible_CNN import Flexible_CNN
from PKLDataset import PKLDataset
from torch.utils.data import DataLoader
import yaml
import random
from tqdm import tqdm

In [3]:
# 设置随机种子
seed = 44
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# 加载配置
with open("../configs/default.yaml", 'r') as f:
    config = yaml.safe_load(f)['baseline']

batch_size = config['batch_size']
num_layers = config['num_layers']
kernel_size = config['kernel_size']
start_channels = config['start_channels']

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
file = ['../datasets/HC_T197_RP.txt','../datasets/HC_T194_RP.txt','../datasets/HC_T191_RP.txt','../datasets/HC_T188_RP.txt','../datasets/HC_T185_RP.txt']
for item in file:
    # 加载测试数据
    test_dataset = PKLDataset(item)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # 初始化模型
    model = Flexible_CNN(
        num_layers=num_layers,
        start_channels=start_channels,
        kernel_size=kernel_size,
        cnn_act='leakrelu',
        num_classes=10
    ).to(device)

    # 加载模型权重
    model_path = 'model/best_model.pth'
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # 定义损失函数
    criterion = nn.CrossEntropyLoss()

    # 评估模型
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            outputs = outputs[0] if isinstance(outputs, tuple) else outputs
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    test_loss /= total
    test_accuracy = correct / total

    print(f"\nfile:{item} \nTest Loss: {test_loss:.6f} Test Accuracy: {test_accuracy:.4f}")

C:\Users\15185\AppData\Local\Temp\ipykernel_23876\2279357325.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d


file:../datasets/HC_T197_RP.txt 
Test Loss: 3.081436 Test Accuracy: 0.5663


Testing: 100%|██████████| 304/304 [00:05<00:00, 60.28it/s]



file:../datasets/HC_T194_RP.txt 
Test Loss: 3.823188 Test Accuracy: 0.3726


Testing: 100%|██████████| 314/314 [00:05<00:00, 60.17it/s]



file:../datasets/HC_T191_RP.txt 
Test Loss: 3.693304 Test Accuracy: 0.4461


Testing: 100%|██████████| 313/313 [00:05<00:00, 60.37it/s]



file:../datasets/HC_T188_RP.txt 
Test Loss: 3.450329 Test Accuracy: 0.4876


Testing: 100%|██████████| 309/309 [00:05<00:00, 59.44it/s]


file:../datasets/HC_T185_RP.txt 
Test Loss: 3.251379 Test Accuracy: 0.4448
